In [1]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


In [2]:
#imports 
import os
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
from src.baseline.noise import Noise
from src.utils import plot_image, plot_im, save_im

from src.baseline.data_loader import get_loader, get_mean, get_std
from src.baseline.vocabulary import Vocabulary


In [3]:
data_location='/home/space/datasets/COA/generated-single-simple/'
root_folder_images = data_location + '/res_images224x224/'
caption_file  = data_location + '/captions_psumsq.txt224x224'


df = pd.read_csv(caption_file)

train_annotation_file = data_location + '/train_captions_psumsq.txt224x224'




In [4]:
NUM_WORKER = 2
freq_threshold = 5
batch_size = 3

# 30 minutes to create those, as it's baseline, i ran it several times and it's the same
vocab = Vocabulary(freq_threshold)
vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'moline': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33}
vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11'}


In [5]:
# mean = get_mean(train_dataset, train_loader, 224, 224)
# std = get_std(train_dataset, train_loader, mean, 224, 224)
# mean, std


# Using already calculated mean and std in generated-data-api-single dataset, the mean=0.28803130984306335 and std=0.28803130984306335 @ Time = 09:41:35
mean,std = (torch.tensor(0.288031), torch.tensor(0.2880313))
    

In [6]:
height, width = 224, 224
import random

train_transform_list = [
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.RandomCrop(224),
    T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    T.ColorJitter(brightness=.5, hue=.3)
]

# Use RandomApply to apply the transform randomly to some of the images
# print(random_tran)
# print('00000000000')
transform_with_random = T.Compose([
    T.Resize((height, width)), # mandetory                  
    random.choice(train_transform_list),
    T.ToTensor(),
    T.Normalize(mean, std), # mandetory 
    Noise(0.1, 0.05), # this should come at the end
])

# print(transform_with_random)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader, train_dataset = get_loader(
    root_folder=root_folder_images,
    annotation_file=train_annotation_file,
    transform=transform_with_random,
    batch_size=batch_size,
    num_workers=0,
    vocab=vocab,
    device=device,
    pin_memory=False
)

for idx, (imgs, correct_caps,_,_,image_file_names) in enumerate(iter(train_loader)):
    print(image_file_names)
    images, captions = imgs.to(device), correct_caps.to(device)
    print('-----------')
#     break


['image_79.png', 'image_123.png', 'image_203.png']
-----------
['image_145.png', 'image_291.png', 'image_0.png']
-----------
['image_149.png', 'image_223.png', 'image_163.png']
-----------
['image_215.png', 'image_348.png', 'image_335.png']
-----------
['image_65.png', 'image_241.png', 'image_293.png']
-----------
['image_27.png', 'image_63.png', 'image_124.png']
-----------
['image_47.png', 'image_153.png', 'image_329.png']
-----------
['image_109.png', 'image_19.png', 'image_164.png']
-----------
['image_102.png', 'image_354.png', 'image_342.png']
-----------
['image_308.png', 'image_142.png', 'image_269.png']
-----------
['image_21.png', 'image_191.png', 'image_315.png']
-----------
['image_262.png', 'image_88.png', 'image_286.png']
-----------
['image_144.png', 'image_22.png', 'image_143.png']
-----------
['image_91.png', 'image_141.png', 'image_351.png']
-----------
['image_254.png', 'image_252.png', 'image_35.png']
-----------
['image_186.png', 'image_278.png', 'image_222.png']
-